# 手写数字识别

## 加载相关库

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


## 加载相关数据

In [2]:
(train_data, test_data) = datasets.mnist.load_data()
print('train_dataset: ', train_data[0].shape, train_data[1].shape)
print('test_dataset: ', test_data[0].shape, test_data[1].shape)

train_dataset:  (60000, 28, 28) (60000,)
test_dataset:  (10000, 28, 28) (10000,)


## 建模

### DNN

In [19]:
train_x = tf.convert_to_tensor(train_data[0], dtype=tf.float32) / 255. 
test_x = tf.convert_to_tensor(test_data[0], dtype=tf.float32) / 255.

train_y = tf.convert_to_tensor(train_data[1])
test_y = tf.convert_to_tensor(test_data[1], dtype=tf.float32)

train_x = tf.reshape(train_x, (-1, 28*28))
test_x = tf.reshape(test_x, (-1, 28*28))


train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(100)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(100)

In [20]:
class DNN(tf.keras.Model):
    def __init__(self):
        super(DNN, self).__init__()
        self.linear_1 = layers.Dense(512, activation='relu')
        self.linear_2 = layers.Dense(256, activation='relu')
        self.linear_3 = layers.Dense(10)

    def call(self, x):
        x = self.linear_1(x)
        x = self.linear_2(x)
        x = self.linear_3(x)
        return x

model = DNN()

In [21]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam()

In [22]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [24]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_function(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)


@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_function(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
        

In [26]:
EPOCHS = 10


for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_db:
        train_step(images, labels)

    for test_images, test_labels in test_db:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}%, Test Loss: {}, Test Accuracy: {}%'
        
    print(template.format(epoch + 1,
                          train_loss.result(), 
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
        

Epoch 1, Loss: 0.06419120728969574, Accuracy: 98.03333282470703%, Test Loss: 0.10958830267190933, Test Accuracy: 96.43000030517578%
Epoch 2, Loss: 0.04327501356601715, Accuracy: 98.61833190917969%, Test Loss: 0.12394493073225021, Test Accuracy: 96.34000396728516%
Epoch 3, Loss: 0.029262429103255272, Accuracy: 99.09000396728516%, Test Loss: 0.08454370498657227, Test Accuracy: 97.61000061035156%
Epoch 4, Loss: 0.025354649871587753, Accuracy: 99.16333770751953%, Test Loss: 0.10469432920217514, Test Accuracy: 97.25%
Epoch 5, Loss: 0.025377264246344566, Accuracy: 99.15333557128906%, Test Loss: 0.0848904624581337, Test Accuracy: 97.68000030517578%
Epoch 6, Loss: 0.01742452196776867, Accuracy: 99.41000366210938%, Test Loss: 0.07742539048194885, Test Accuracy: 98.1500015258789%
Epoch 7, Loss: 0.012309685349464417, Accuracy: 99.59333801269531%, Test Loss: 0.10285089910030365, Test Accuracy: 97.59000396728516%
Epoch 8, Loss: 0.014455433003604412, Accuracy: 99.50666809082031%, Test Loss: 0.082372

In [9]:
model.summary()

Model: "dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  401920    
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
dense_2 (Dense)              multiple                  2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [32]:
train_x = tf.convert_to_tensor(train_data[0], dtype=tf.float32) / 255. 
test_x = tf.convert_to_tensor(test_data[0], dtype=tf.float32) / 255.

train_y = tf.convert_to_tensor(train_data[1])
test_y = tf.convert_to_tensor(test_data[1], dtype=tf.float32)

train_x = tf.reshape(train_x, (-1, 28, 28, 1))
test_x = tf.reshape(test_x, (-1, 28, 28, 1))


train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(100)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(100)

In [33]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv_1 = layers.Conv2D(32, (3, 3), activation='relu')
        self.conv_2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.conv_3 = layers.Conv2D(64, (3, 3), activation='relu')
        self.linear_1 = layers.Dense(64, activation='relu')
        self.linear_2 = layers.Dense(10)
        self.pooling = layers.MaxPool2D((2, 2))
        self.flatten = layers.Flatten()

    def call(self, x):
        x = self.conv_1(x)
        x = self.pooling(x)
        x = self.conv_2(x)
        x = self.pooling(x)
        x = self.conv_3(x)
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

model = CNN()

In [34]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_function(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)


@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_function(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [35]:
EPOCHS = 10


for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_db:
        train_step(images, labels)

    for test_images, test_labels in test_db:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {:.2}, Accuracy: {}%, Test Loss: {:.2}, Test Accuracy: {}%'
        
    print(template.format(epoch + 1,
                          train_loss.result(), 
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

2021-08-26 22:02:38.508520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-26 22:02:45.013576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1, Loss: 0.13, Accuracy: 95.91667175292969%, Test Loss: 0.044, Test Accuracy: 98.44000244140625%
Epoch 2, Loss: 0.039, Accuracy: 98.80833435058594%, Test Loss: 0.034, Test Accuracy: 98.79000091552734%
Epoch 3, Loss: 0.027, Accuracy: 99.16999816894531%, Test Loss: 0.032, Test Accuracy: 98.77000427246094%
Epoch 4, Loss: 0.019, Accuracy: 99.41666412353516%, Test Loss: 0.033, Test Accuracy: 98.95000457763672%
Epoch 5, Loss: 0.014, Accuracy: 99.59833526611328%, Test Loss: 0.041, Test Accuracy: 98.77000427246094%
Epoch 6, Loss: 0.012, Accuracy: 99.63166809082031%, Test Loss: 0.043, Test Accuracy: 98.80000305175781%
Epoch 7, Loss: 0.011, Accuracy: 99.67832946777344%, Test Loss: 0.042, Test Accuracy: 98.81000518798828%
Epoch 8, Loss: 0.0095, Accuracy: 99.69499969482422%, Test Loss: 0.045, Test Accuracy: 98.9000015258789%
Epoch 9, Loss: 0.0084, Accuracy: 99.7316665649414%, Test Loss: 0.043, Test Accuracy: 98.9000015258789%
Epoch 10, Loss: 0.0072, Accuracy: 99.7683334350586%, Test Loss: 0.

In [ ]:
model.summary()